# 导包

In [ ]:
import torch
import torch.nn as nn
from torch.nn import init
import torch.optim as optim
import torch.utils.data as Data

from torchtext import data
from torchtext.vocab import Vectors
from torchtext.data import Iterator, BucketIterator

from sklearn.model_selection import train_test_split

import csv
import numpy as np
import pandas as pd

from google.colab import drive
import time
from tqdm import tqdm

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 定义超参数

In [ ]:
BATCH_SIZE = 8
EPOCHS = 10
FIX_LENGTH = 2500
N_CLASS=14
LR=0.001
OUT_CHANNEL=1000
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 划分训练集验证集

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/data/News classification/train_set.csv', sep='\t')

train, val = train_test_split(train_df, test_size=0.1)
train.to_csv("/content/drive/MyDrive/data/News classification/tmp/train.csv", index=False)
val.to_csv("/content/drive/MyDrive/data/News classification/tmp/val.csv", index=False)

# 构建Dataset

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/data/News classification/tmp/train.csv')
valid_data = pd.read_csv('/content/drive/MyDrive/data/News classification/tmp/val.csv')
test_data = pd.read_csv("/content/drive/MyDrive/data/News classification/test_a.csv")

tokenize = lambda x: x.split()
# fix_length指定了每条文本的长度，截断补长
TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True, fix_length=FIX_LENGTH)
LABEL = data.Field(sequential=False, use_vocab=False)

def get_dataset(csv_data, text_field, label_field, test=False):
	# id数据对训练在训练过程中没用，使用None指定其对应的field
    fields = [("id", None), # we won't be needing the id, so we pass in None as the field
                 ("text", text_field), ("label", label_field)]       
    examples = []

    if test:
        # 如果为测试集，则不加载label
        for text in tqdm(csv_data['text']):
            examples.append(data.Example.fromlist([None, text, None], fields))
    else:
        for text, label in tqdm(zip(csv_data['text'], csv_data['label'])):
            examples.append(data.Example.fromlist([None, text, label], fields))
    return examples, fields


# 得到构建Dataset所需的examples和fields
train_examples, train_fields = get_dataset(train_data, TEXT, LABEL)
valid_examples, valid_fields = get_dataset(valid_data, TEXT, LABEL)
test_examples, test_fields = get_dataset(test_data, TEXT, None, test=True)

# 构建Dataset数据集
train = data.Dataset(train_examples, train_fields)
valid = data.Dataset(valid_examples, valid_fields)
test = data.Dataset(test_examples, test_fields)

180000it [00:56, 3171.83it/s]
20000it [00:05, 3639.63it/s]
100%|██████████| 50000/50000 [00:17<00:00, 2933.10it/s]


# 构建迭代器

In [ ]:
train_iter, val_iter = BucketIterator.splits(
        (train, valid),
        batch_sizes=(BATCH_SIZE, BATCH_SIZE),
        device = DEVICE, # 如果使用gpu，此处将-1更换为GPU的编号
        sort_key=lambda x: len(x.text), # the BucketIterator needs to be told what function it should use to group the data.
        sort_within_batch=False,
        repeat=False # we pass repeat=False because we want to wrap this Iterator layer.
)

test_iter = Iterator(test, batch_size=BATCH_SIZE, device=DEVICE, sort=False, train=False, sort_within_batch=False, repeat=False) # train=False可以保证顺序不变

# 加载词向量

In [ ]:
vectors = Vectors(name='/content/drive/MyDrive/data/News classification/tmp/mymodel.txt')
TEXT.build_vocab(train, vectors=vectors)
weight_matrix = TEXT.vocab.vectors

 96%|█████████▌| 5894/6151 [00:00<00:00, 18770.32it/s]


# 定义模型

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.embedding = nn.Embedding(len(TEXT.vocab), 100)     
        self.embedding.weight.data.copy_(weight_matrix)

        self.conv = nn.Sequential(
            # (8, 1, 2500, 100)
            nn.Conv2d(1, OUT_CHANNEL, (2, 100)), # input_channel(=1), output_channel, (filter_height, filter_width), stride=1]
            # (8, 1000, 2499, 100)
            nn.ReLU(),
            nn.MaxPool2d((2499, 1)), # ((filter_height, filter_width))
            # (8, 1000, 1, 1)
        )
        self.out = nn.Linear(OUT_CHANNEL, N_CLASS)

    def forward(self, X): # X: [fix_length, batch_size]
        embeds = self.embedding(X)  # [fix_length, batch_size, embedding_size]  
        embeds = self.embedding(X).transpose(0, 1) # [batch_size, sequence_length, embedding_size]
        embeds = embeds.unsqueeze(1) # [batch, channel(=1), sequence_length, embedding_size]
        
        conved = self.conv(embeds)
        flatten = conved.view(BATCH_SIZE, -1)
        y = self.out(flatten)
        return y

model = CNN().to(DEVICE)
loss_fc = nn.CrossEntropyLoss().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LR)

# 训练

In [ ]:
start = time.perf_counter()
for epoch in range(EPOCHS):
    for i, batch in enumerate(train_iter):
        optimizer.zero_grad()
        pred = model(batch.text)
	
        loss = loss_fc(pred, batch.label)
        loss.backward()
        optimizer.step()
        if (i+1) % 1250 == 0:
            end = time.perf_counter()
            print('Epoch: ', epoch, '| batch: ', (i+1)*8, '| train loss: %.4f' % loss, '| time: %.2f s' % (end-start))


Epoch:  0 | batch:  10000 | train loss: 0.6136 | time: 31.84 s
Epoch:  0 | batch:  20000 | train loss: 0.1008 | time: 63.23 s
Epoch:  0 | batch:  30000 | train loss: 0.9332 | time: 94.60 s
Epoch:  0 | batch:  40000 | train loss: 0.2081 | time: 125.95 s
Epoch:  0 | batch:  50000 | train loss: 0.1780 | time: 157.31 s
Epoch:  0 | batch:  60000 | train loss: 0.9344 | time: 188.67 s
Epoch:  0 | batch:  70000 | train loss: 0.0056 | time: 220.07 s
Epoch:  0 | batch:  80000 | train loss: 0.6538 | time: 251.66 s
Epoch:  0 | batch:  90000 | train loss: 0.0562 | time: 283.16 s
Epoch:  0 | batch:  100000 | train loss: 0.0129 | time: 314.52 s
Epoch:  0 | batch:  110000 | train loss: 0.2322 | time: 345.95 s
Epoch:  0 | batch:  120000 | train loss: 0.0278 | time: 377.38 s
Epoch:  0 | batch:  130000 | train loss: 0.8838 | time: 408.77 s
Epoch:  0 | batch:  140000 | train loss: 0.3172 | time: 440.15 s
Epoch:  0 | batch:  150000 | train loss: 0.3538 | time: 471.53 s
Epoch:  0 | batch:  160000 | train lo

# 评估

In [ ]:
true = 0.0
all = 0.0
for i, val_batch in enumerate(val_iter):
    pred_y = torch.max(model(val_batch.text), 1)[1].cpu().data.numpy()
    real_y = val_batch.label.cpu().data.numpy()
    true += float((pred_y == real_y).astype(int).sum())
    all += float(len(real_y))
accuracy = true / all
print(accuracy)


0.93295


# 预测

In [ ]:
result=[['label']]
for i, test_batch in enumerate(test_iter):
    for label in torch.max(model(test_batch.text), 1)[1].cpu().data.numpy():
       result.append([label]) 
with open("/content/drive/MyDrive/data/News classification/tmp/result.csv", "a", newline='', encoding='utf-8') as file:
    writer = csv.writer(file ,delimiter=',')
    writer.writerows(result)